In [ ]:
!pip install transformers datasets evaluate rouge_score bert-score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests
import json
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
  DistilBertTokenizerFast,
  DistilBertForSequenceClassification,
  Trainer,
  TrainingArguments,
  set_seed
)
import torch
import evaluate

urls = {
  "train": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/Training/train.json",
  "validation": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/practice_data/practice.json",
  "test": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/test/FMD_test.json"
}

def fetch_dataset(url):
  r = requests.get(url)
  r.raise_for_status()
  lines = r.text.strip().split("\n")
  data = [json.loads(line) for line in lines]
  return Dataset.from_list(data)

dataset = DatasetDict({k: fetch_dataset(v) for k, v in urls.items()})

label_mapping = {"True": 0, "False": 1, "NEI": 2}

def preprocess(example):
  sci_text = ""
  if "sci_digest" in example and isinstance(example["sci_digest"], list) and len(example["sci_digest"]) > 0:
      sci_text = example["sci_digest"][0]
  claim_text = example.get("claim", "")
  justification_text = example.get("justification", "")
  evidence_text = example.get("evidence", "")
  example["text"] = (
      f"Claim: {claim_text} "
      f"Justification: {justification_text} "
      f"Scientific Digest: {sci_text} "
      f"Evidence: {evidence_text}"
  ).strip()
  if "label" in example and example["label"] in label_mapping:
      example["label_id"] = label_mapping[example["label"]]
  return example

dataset["train"] = dataset["train"].map(preprocess)
dataset["validation"] = dataset["validation"].map(preprocess)
dataset["test"] = dataset["test"].map(preprocess)

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_fn(examples):
  tok_output = tokenizer(examples["text"], truncation=True, max_length=256)
  if "label_id" in examples:
      tok_output["labels"] = examples["label_id"]
  return tok_output

train_data = dataset["train"].map(tokenize_fn, batched=True, remove_columns=dataset["train"].column_names)
val_data = dataset["validation"].map(tokenize_fn, batched=True, remove_columns=dataset["validation"].column_names)
test_data = dataset["test"].map(tokenize_fn, batched=True, remove_columns=dataset["test"].column_names)

set_seed(42)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

training_args = TrainingArguments(
  output_dir="finfact_model",
  evaluation_strategy="epoch",
  save_strategy="epoch",
  num_train_epochs=10,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  logging_steps=50,
  learning_rate=1e-4,
  seed=42,
  load_best_model_at_end=True  # Optional: Load the best model based on validation performance
)

metric_accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  return metric_accuracy.compute(predictions=preds, references=labels)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_data,
  eval_dataset=val_data,
  tokenizer=tokenizer,
  compute_metrics=compute_metrics
)

# Training and Validation
trainer.train()

# Final Validation Metrics
metrics = trainer.evaluate()
validation_accuracy = metrics["eval_accuracy"]  # Extract validation accuracy
print("Validation Accuracy:", validation_accuracy)

# Testing
predictions = trainer.predict(test_data)
print("Test logits shape:", predictions.predictions.shape)

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1304 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.905400,0.855921,0.626667
2,0.825800,0.731654,0.625000
3,0.694400,0.680659,0.770000
4,0.524500,0.471511,0.806667
5,0.418000,0.464280,0.861667
6,0.324700,0.375774,0.901667
7,0.247300,0.330867,0.930000
8,0.178400,0.304481,0.931667
9,0.138100,0.276568,0.948333
10,0.185500,0.256632,0.950000


Validation Accuracy: 0.95
Test logits shape: (1304, 3)


In [ ]:
import requests
import json
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    set_seed
)
import torch
import evaluate

# URLs for dataset
urls = {
    "train": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/Training/train.json",
    "validation": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/practice_data/practice.json",
    "test": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/test/FMD_test.json"
}

# Fetch dataset from URLs
def fetch_dataset(url):
    r = requests.get(url)
    r.raise_for_status()
    lines = r.text.strip().split("\n")
    data = [json.loads(line) for line in lines]
    return Dataset.from_list(data)

# Load datasets
dataset = DatasetDict({k: fetch_dataset(v) for k, v in urls.items()})

# Label mapping
label_mapping = {"True": 0, "False": 1, "NEI": 2}

# Preprocessing function
def preprocess(example):
    sci_text = ""
    if "sci_digest" in example and isinstance(example["sci_digest"], list) and len(example["sci_digest"]) > 0:
        sci_text = example["sci_digest"][0]
    example["text"] = (
        f"Claim: {example.get('claim', '')} "
        f"Justification: {example.get('justification', '')} "
        f"Scientific Digest: {sci_text} "
        f"Evidence: {example.get('evidence', '')}"
    ).strip()
    if "label" in example and example["label"] in label_mapping:
        example["label_id"] = label_mapping[example["label"]]
    return example

# Apply preprocessing
for split in dataset:
    dataset[split] = dataset[split].map(preprocess)

# Tokenizer and model
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# Tokenization function
def tokenize_fn(examples):
    tok_output = tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )
    if "label_id" in examples:
        tok_output["labels"] = examples["label_id"]
    return tok_output

# Tokenize datasets
train_data = dataset["train"].map(tokenize_fn, batched=True, remove_columns=dataset["train"].column_names)
val_data = dataset["validation"].map(tokenize_fn, batched=True, remove_columns=dataset["validation"].column_names)
test_data = dataset["test"].map(tokenize_fn, batched=True, remove_columns=dataset["test"].column_names)

# Set seed for reproducibility
set_seed(42)

# Training arguments
training_args = TrainingArguments(
    output_dir="roberta_finfact_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,  # Accumulate gradients for smaller batches
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    seed=42,
    fp16=True,  # Mixed precision for faster training
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1"
)

# Metrics
metric_accuracy = evaluate.load("accuracy")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": metric_accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": metric_precision.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": metric_recall.compute(predictions=preds, references=labels, average="macro")["recall"],
        "f1": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Training
trainer.train()

# Evaluation
metrics = trainer.evaluate()
print(f"Validation Metrics: {metrics}")

# Testing
predictions = trainer.predict(test_data)
preds = np.argmax(predictions.predictions, axis=-1)
print(f"Test Predictions Shape: {predictions.predictions.shape}")
print(f"Sample Test Predictions: {preds[:10]}")

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1304 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-4-87e610356633>:114: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.799362,0.645000,0.448909,0.523939,0.474659
2,1.765200,0.668733,0.691667,0.747540,0.570332,0.531248
3,1.765200,0.499292,0.805000,0.794347,0.744942,0.754480
4,1.274800,0.342533,0.873333,0.853583,0.841047,0.846496
5,0.827100,0.322837,0.885000,0.900446,0.842681,0.861345
6,0.827100,0.261136,0.920000,0.924285,0.892639,0.904002
7,0.493100,0.204159,0.945000,0.935959,0.938178,0.937030
8,0.493100,0.222873,0.930000,0.916803,0.918920,0.917767
9,0.309900,0.239419,0.933333,0.934619,0.911074,0.921212
10,0.242500,0.208283,0.953333,0.946260,0.948593,0.947387


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Metrics: {'eval_loss': 0.2471693456172943, 'eval_accuracy': 0.9633333333333334, 'eval_precision': 0.9571174282549494, 'eval_recall': 0.9586185460306466, 'eval_f1': 0.9577959801423489, 'eval_runtime': 2.3971, 'eval_samples_per_second': 250.3, 'eval_steps_per_second': 15.852, 'epoch': 19.682926829268293}
Test Predictions Shape: (1304, 3)
Sample Test Predictions: [0 1 0 1 2 1 1 2 1 2]


RoBERTa

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

Evidence Generation

Classification


In [ ]:
import requests
import json
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    set_seed
)
import torch
import evaluate
from sklearn.metrics import precision_score, recall_score, f1_score


# URLs for dataset
urls = {
    "train": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/Training/train.json",
    "validation": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/practice_data/practice.json",
    "test": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/test/FMD_test.json"
}

# Fetch dataset from URLs
def fetch_dataset(url):
    r = requests.get(url)
    r.raise_for_status()
    lines = r.text.strip().split("\n")
    data = [json.loads(line) for line in lines]
    return Dataset.from_list(data)

# Load datasets
dataset = DatasetDict({k: fetch_dataset(v) for k, v in urls.items()})

# Label mapping
label_mapping = {"True": 0, "False": 1, "NEI": 2}

# Preprocessing function
def preprocess(example):
    sci_text = ""
    if "sci_digest" in example and isinstance(example["sci_digest"], list) and len(example["sci_digest"]) > 0:
        sci_text = example["sci_digest"][0]
    example["text"] = (
        f"Claim: {example.get('claim', '')} "
        f"Justification: {example.get('justification', '')} "
        f"Scientific Digest: {sci_text} "
    ).strip()
    if "label" in example and example["label"] in label_mapping:
        example["label_id"] = label_mapping[example["label"]]
    return example

# Apply preprocessing
for split in dataset:
    dataset[split] = dataset[split].map(preprocess)

# Tokenizer and model
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# Tokenization function
def tokenize_fn(examples):
    tok_output = tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )
    if "label_id" in examples:
        tok_output["labels"] = examples["label_id"]
    return tok_output

# Tokenize datasets
train_data = dataset["train"].map(tokenize_fn, batched=True, remove_columns=dataset["train"].column_names)
val_data = dataset["validation"].map(tokenize_fn, batched=True, remove_columns=dataset["validation"].column_names)
test_data = dataset["test"].map(tokenize_fn, batched=True, remove_columns=dataset["test"].column_names)

# Set seed for reproducibility
set_seed(42)

# Training arguments
training_args = TrainingArguments(
    output_dir="roberta_finfact_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,  # Accumulate gradients for smaller batches
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=100,
    seed=42,
    fp16=True,  # Mixed precision for faster training
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1"
)

# Metrics
metric_accuracy = evaluate.load("accuracy")
metric_precision = evaluate.load("precision")
metric_recall = evaluate.load("recall")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision = precision_score(labels, preds, average="macro", zero_division=0)
    recall = recall_score(labels, preds, average="macro", zero_division=0)
    f1 = f1_score(labels, preds, average="macro", zero_division=0)
    accuracy = metric_accuracy.compute(predictions=preds, references=labels)["accuracy"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Training
trainer.train()

# Evaluation
metrics = trainer.evaluate()
print(f"Validation Metrics: {metrics}")

# Testing
predictions = trainer.predict(test_data)
preds = np.argmax(predictions.predictions, axis=-1)
print(f"Test Predictions Shape: {predictions.predictions.shape}")
print(f"Sample Test Predictions: {preds[:10]}")

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1304 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1304 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-993dcd3c67f2>:114: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.837100,0.728000,0.660000,0.478985,0.535351,0.489812
2,1.445700,0.555266,0.765000,0.746065,0.661061,0.643104
3,1.266400,0.405244,0.838333,0.809073,0.809582,0.809317
4,1.056400,0.291687,0.893333,0.877991,0.873274,0.875445
5,0.633000,0.273159,0.916667,0.906065,0.902789,0.903381
6,0.450400,0.226963,0.936667,0.935863,0.922656,0.928587
7,0.350800,0.226148,0.946667,0.940948,0.937197,0.938888
8,0.233400,0.276714,0.936667,0.930002,0.928697,0.929304
9,0.081500,0.289663,0.948333,0.942963,0.940980,0.941932


Validation Metrics: {'eval_loss': 0.28966328501701355, 'eval_accuracy': 0.9483333333333334, 'eval_precision': 0.9429627133031245, 'eval_recall': 0.9409798899633892, 'eval_f1': 0.9419319233447122, 'eval_runtime': 2.6016, 'eval_samples_per_second': 230.625, 'eval_steps_per_second': 28.828, 'epoch': 9.922448979591836}
Test Predictions Shape: (1304, 3)
Sample Test Predictions: [0 1 0 1 2 1 1 0 1 2]


Evidence generation

In [ ]:
import requests
import json
from datasets import Dataset, DatasetDict
import torch

from transformers import (
    T5TokenizerFast,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed
)

# ---------------------------------------------------------
# 1. Fetch the Dataset
# ---------------------------------------------------------
urls = {
    "train": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/Training/train.json",
    "validation": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/practice_data/practice.json",
    "test": "https://huggingface.co/datasets/lzw1008/COLING25-FMD/resolve/main/test/FMD_test.json"
}

def fetch_dataset(url):
    r = requests.get(url)
    r.raise_for_status()
    lines = r.text.strip().split("\n")
    data = [json.loads(line) for line in lines]
    return Dataset.from_list(data)

dataset = DatasetDict({split: fetch_dataset(url) for split, url in urls.items()})

# ---------------------------------------------------------
# 2. Preprocess: Build a Prompt and Target (Evidence)
# ---------------------------------------------------------
def preprocess(example):
    # Read 'sci_digest' safely
    sci_text = ""
    if "sci_digest" in example and isinstance(example["sci_digest"], list) and len(example["sci_digest"]) > 0:
        sci_text = example["sci_digest"][0]

    # We have these fields:
    # - "claim"
    # - "justification"
    # - "sci_digest" (above)
    # - "evidence" (present in train/val, absent in test)

    claim_text = example.get("claim", "")
    justification_text = example.get("justification", "")
    # If "evidence" is missing or empty, use "N/A"
    evidence_text = example.get("evidence", "N/A")

    # Combine the relevant parts into a single prompt
    # We'll instruct the model to produce the evidence.
    prompt_str = (
        "Task: Generate the evidence that supports (or refutes) the claim.\n\n"
        f"Claim: {claim_text}\n"
        f"Justification: {justification_text}\n"
        f"Scientific Digest: {sci_text}\n\n"
        "Evidence:"
    )

    # Use the dataset's 'evidence' as the ground truth for training (if present).
    # In the test set, there's no 'evidence', so it remains "N/A".
    example["prompt"] = prompt_str
    example["target"] = evidence_text

    return example

for split in dataset.keys():
    dataset[split] = dataset[split].map(preprocess)

# ---------------------------------------------------------
# 3. Tokenize Data for T5
# ---------------------------------------------------------
model_checkpoint = "google/t5-efficient-tiny"  # a very small T5 checkpoint
tokenizer = T5TokenizerFast.from_pretrained(model_checkpoint)

def tokenize_fn(examples):
    # Tokenize the prompt (input)
    model_inputs = tokenizer(
        examples["prompt"],
        max_length=256,
        truncation=True,
        padding="max_length"
    )
    # Tokenize the target (evidence) with the T5 target tokenizer context
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Remove original columns to keep only the tokenized data
columns_to_remove = list(set(dataset["train"].column_names))  # includes 'claim', 'evidence', etc.
columns_to_remove_test = list(set(dataset["test"].column_names))  # includes 'claim', 'evidence', etc.

train_data = dataset["train"].map(tokenize_fn, batched=True, remove_columns=columns_to_remove)
val_data   = dataset["validation"].map(tokenize_fn, batched=True, remove_columns=columns_to_remove)
test_data  = dataset["test"].map(tokenize_fn, batched=True, remove_columns=columns_to_remove_test)

# ---------------------------------------------------------
# 4. Create T5 Model and TrainingArguments
# ---------------------------------------------------------
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
set_seed(42)

training_args = Seq2SeqTrainingArguments(
    output_dir="t5_evidence_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,            # fewer epochs for demo
    per_device_train_batch_size=2, # reduce batch size to avoid OOM
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4, # accumulate gradients to emulate larger batch
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    seed=42,
    fp16=True,                     # Mixed-precision (if supported)
    load_best_model_at_end=True,
    # No classification metrics are computed, only text generation
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=None  # We won't compute any textual metrics here
)

# ---------------------------------------------------------
# 5. Train
# ---------------------------------------------------------
trainer.train()

# Evaluate on Validation (loss-based metrics only)
val_metrics = trainer.evaluate()
print("\nValidation metrics (loss only):", val_metrics)

# ---------------------------------------------------------
# 6. Test: Generate Evidence
# ---------------------------------------------------------
test_outputs = trainer.predict(test_data)
print("\nTest metrics (loss only, no 'evidence' in test set):", test_outputs.metrics)

# Let's decode some sample predictions from the test set
# We'll see what "evidence" the model comes up with
sample_indices = [0, 1, 2]  # just 3 samples
input_ids = torch.tensor([test_data[i]["input_ids"] for i in sample_indices]).to(model.device)
attention_mask = torch.tensor([test_data[i]["attention_mask"] for i in sample_indices]).to(model.device)

generated = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=50
)

decoded_preds = tokenizer.batch_decode(generated, skip_special_tokens=True)

for idx, pred in zip(sample_indices, decoded_preds):
    print(f"\n--- Test Sample {idx} ---")
    print("Prompt:\n", tokenizer.decode(test_data[idx]["input_ids"], skip_special_tokens=True))
    print("Model Generated Evidence:\n", pred)

Map:   0%|          | 0/1953 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1304 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-adb775df437c>:128: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.000000,nan



Validation metrics (loss only): {'eval_loss': nan, 'eval_runtime': 5.2667, 'eval_samples_per_second': 113.923, 'eval_steps_per_second': 56.962, 'epoch': 1.9948822927328558}


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.49 GiB. GPU 0 has a total capacity of 14.75 GiB of which 5.44 GiB is free. Process 15478 has 9.30 GiB memory in use. Of the allocated memory 7.66 GiB is allocated by PyTorch, and 1.51 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)